In [1]:
import asyncio
from playwright.async_api import async_playwright, expect
import time

import os
from dotenv import load_dotenv

import random

load_dotenv()

True

In [2]:
import pandas as pd

df = pd.read_csv("../Data/LGAs.csv")

lgas = df["LGA (EN)"].tolist()

In [3]:
button_titles = {
    "Row" : "Add selected items to the table row",
    "Column" : "Add selected items to the table column",
    "Wafer": "Add selected items to the table wafer",
    "Filter": "Add selected items to the table filter"
}

iterable_var = {
    "subdirs" : ["Geographical Areas (Enumeration)"],
    "name" : "Local Government Areas (2022 Boundaries) (EN)",
    "level" : "LGA (EN)",
    "add to" : "Filter"
}

table = {
    "subdirs" : ["2021 Census of Population and Housing", "Census TableBuilder Pro"],
    "name" : "2021 Census - counting persons, place of enumeration",
    "saved" : "base_table"
}

const_vars = {}

In [4]:
states = ["New South Wales", "Victoria", "Queensland", "South Australia", "Western Australia", "Tasmania", "Northern Territory", "Australian Capital Territory"]

spare = "SEIFA Deciles (Population-based) WARNING: Population deciles must be used with same geography type"

In [5]:
queued = []

In [6]:
async def select_const_vars(page):
    subdirs = table["subdirs"].copy()

    for subdir in subdirs:
        await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(subdir, exact=True)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

    await page.get_by_title(table["name"]).click()

    await page.get_by_title(table["saved"]).dblclick()

    return 

In [ ]:
async def scrape(playwright, lga):
    browser = await playwright.chromium.launch(headless=False)
    page = await browser.new_page()
    await page.goto('https://tablebuilder.abs.gov.au/webapi/jsf/login.xhtml')

    await page.get_by_label("User ID is a number:").fill(os.getenv("ABS_TABLEBUILDER_USERNAME"))
    await page.get_by_label("Password:").fill(os.getenv("ABS_TABLEBUILDER_PASSWORD"))
    await page.get_by_role("button", name="Log in").click()

    await page.wait_for_load_state()

    await select_const_vars(page)

    #await page.wait_for_load_state(time)

    await expect(page.locator("//*[contains(@id, 'showSchemaPane')]", has = page.get_by_text("Collapse all"))).to_be_visible(timeout=600_000)

    await page.get_by_text("Collapse all").click()

    subdirs = iterable_var["subdirs"].copy()

    for subdir in subdirs:
        await page.get_by_title(subdir).click()
        await page.get_by_title(subdir).click()

    await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(iterable_var["name"], exact=True)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()
    
    for state in states:
        await page.locator("//*[contains(@class, 'treeNodeElement')]", has = page.get_by_title(state)).locator("//*[contains(@class, 'treeNodeExpander collapsed')]").click()

    await page.locator("//*[contains(@class, 'treeNodeElementBody')]", has = page.get_by_title(lga)).locator("//*[contains(@type, 'checkbox')]").check()

    await page.get_by_title(button_titles[iterable_var["add to"]]).click()

    await page.wait_for_load_state()

    await page.locator("//*[contains(@id, 'downloadControl:downloadLargeTableButton')]").click()

    await expect(page.locator("//*[contains(@id, 'downloadTableModePanel_shade')]")).to_be_visible(timeout=600_000)

    await page.locator("//*[contains(@id, 'downloadTableModeForm:downloadTableNameTxt')]").fill("LGA_{}".format(lga))

    await page.locator("//*[contains(@id, 'downloadTableModeForm:queueTableButton')]", has = page.get_by_text("Queue table")).click()
    
    await page.wait_for_load_state()


In [8]:
#while len(queued) != 556:
lga = random.choice(lgas)

#if lga in queued:
    #continue

playwright = await async_playwright().start()

try:
    await scrape(playwright, lga)
    queued.append(lga)

    print("Queued: {}".format(lga))

    await playwright.stop()
except Exception as e:
    print("Failed: {}".format(lga))
    print(e)
    time.sleep(5) 

    await playwright.stop()

Failed: East Pilbara
Locator expected to be visible
Actual value: <element(s) not found> 
Call log:
  - LocatorAssertions.to_be_visible with timeout 60000ms
  -   - waiting for locator("//*[contains(@id, 'showSchemaPane')]").filter(has=get_by_text("Collapse all"))

